Fetching data from mongodb Atlas server

In [35]:
from pymongo import MongoClient
import pandas as pd

# clinet = MongoClient("mongodb+srv://himayudhoke:<password>@cluster0.lufwdkw.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0")
# database = clinet["thane_traffic"]
# collection = database["traffic_flow_data"]
# print(collection.count_documents({}))


# document = list(collection.find().limit(1000))
# df = pd.DataFrame(document)
# traffic_data = df.to_csv('Traffic.csv',index=False)

In [36]:
df = pd.read_csv(r'C:\Users\himay\OneDrive\Desktop\NeuroTraff\artifacts\raw_data.csv')
df.head(10)

,_id,road,point,timestamp,roadName,frc,currentSpeed,freeFlowSpeed,currentTravelTime,freeFlowTravelTime,confidence,roadClosure
0,68af0164c0e5f68e3352f663,SH-42 (Ghodbunder Road),"19.2500,73.0500",2025-08-27 13:00:19.873,NaN,FRC1,39,39,918,918,1.000000,False
1,68af0166c0e5f68e3352f664,SH-42 (Ghodbunder Road),"19.2500,72.9800",2025-08-27 13:00:19.873,NaN,FRC4,23,23,190,190,1.000000,False
2,68af0166c0e5f68e3352f665,Eastern Express Highway,"19.2183,72.9781",2025-08-27 13:00:19.873,NaN,FRC4,6,10,6,3,1.000000,False
3,68af0166c0e5f68e3352f666,Eastern Express Highway,"19.0760,72.8777",2025-08-27 13:00:19.873,NaN,FRC4,19,19,148,148,1.000000,False
4,68af0166c0e5f68e3352f667,LBS Marg,"19.1960,72.9600",2025-08-27 13:00:19.873,NaN,FRC4,21,21,228,228,1.000000,False
5,68af0167c0e5f68e3352f668,LBS Marg,"19.0500,72.8700",2025-08-27 13:00:19.873,NaN,FRC2,54,54,126,126,1.000000,False
6,68af0167c0e5f68e3352f669,Thane-Belapur Road,"19.1800,73.0100",2025-08-27 13:00:19.873,NaN,FRC4,32,32,1022,1022,1.000000,False
7,68af0167c0e5f68e3352f66a,Thane-Belapur Road,"19.0450,73.0150",2025-08-27 13:00:19.873,NaN,FRC4,20,20,220,220,1.000000,False
8,68af0168c0e5f68e3352f66b,Sion-Panvel Expressway,"19.0600,72.8850",2025-08-27 13:00:19.873,NaN,FRC4,15,19,1000,789,0.947993,False
9,68af0168c0e5f68e3352f66c,Sion-Panvel Expressway,"18.9800,73.1100",2025-08-27 13:00:19.873,NaN,FRC4,25,30,555,462,0.985374,False


Data Cleaning

In [38]:
# Removing cloumns
df.drop(columns=['_id','roadName'],inplace=True)
df.head()

KeyError: "['_id', 'roadName'] not found in axis"

In [39]:
# label Enconding
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['road'] = le.fit_transform(df['road'])
df['frc'] = le.fit_transform(df['frc'])
df['roadClosure'] = le.fit_transform(df['roadClosure'])
df.head()

,road,point,timestamp,frc,currentSpeed,freeFlowSpeed,currentTravelTime,freeFlowTravelTime,confidence,roadClosure
0,18,"19.2500,73.0500",2025-08-27 13:00:19.873,0,39,39,918,918,1.0,0
1,18,"19.2500,72.9800",2025-08-27 13:00:19.873,3,23,23,190,190,1.0,0
2,4,"19.2183,72.9781",2025-08-27 13:00:19.873,3,6,10,6,3,1.0,0
3,4,"19.0760,72.8777",2025-08-27 13:00:19.873,3,19,19,148,148,1.0,0
4,9,"19.1960,72.9600",2025-08-27 13:00:19.873,3,21,21,228,228,1.0,0


Feature Engineering

In [40]:
#latitude and longitude

df[['latitude','longitude']] = df['point'].str.split(',',expand=True)
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)
df.drop(columns='point',inplace=True)
df.head()

,road,timestamp,frc,currentSpeed,freeFlowSpeed,currentTravelTime,freeFlowTravelTime,confidence,roadClosure,latitude,longitude
0,18,2025-08-27 13:00:19.873,0,39,39,918,918,1.0,0,19.2500,73.0500
1,18,2025-08-27 13:00:19.873,3,23,23,190,190,1.0,0,19.2500,72.9800
2,4,2025-08-27 13:00:19.873,3,6,10,6,3,1.0,0,19.2183,72.9781
3,4,2025-08-27 13:00:19.873,3,19,19,148,148,1.0,0,19.0760,72.8777
4,9,2025-08-27 13:00:19.873,3,21,21,228,228,1.0,0,19.1960,72.9600


In [41]:
import pytz

df['timestamp'] = pd.to_datetime(df['timestamp']).dt.tz_localize('UTC')
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.tz_convert('Asia/Kolkata')
df['timestamp'] = df['timestamp'].dt.tz_localize(None)

df['Day'] = df['timestamp'].dt.day
df['Hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute

df.drop(columns='timestamp',inplace=True)
df.head()

,road,frc,currentSpeed,freeFlowSpeed,currentTravelTime,freeFlowTravelTime,confidence,roadClosure,latitude,longitude,Day,Hour,minute
0,18,0,39,39,918,918,1.0,0,19.2500,73.0500,27,18,30
1,18,3,23,23,190,190,1.0,0,19.2500,72.9800,27,18,30
2,4,3,6,10,6,3,1.0,0,19.2183,72.9781,27,18,30
3,4,3,19,19,148,148,1.0,0,19.0760,72.8777,27,18,30
4,9,3,21,21,228,228,1.0,0,19.1960,72.9600,27,18,30


In [42]:
#delay 
import numpy as np
df['Delay'] = (df['currentTravelTime'] - df['freeFlowTravelTime'])

# Avoid division by zero or NaNs
df['delay ratio'] = np.where(
    df['freeFlowTravelTime'] > 0,
    df['Delay'] / df['freeFlowTravelTime'],
    np.nan
)

df.head()

,road,frc,currentSpeed,freeFlowSpeed,currentTravelTime,freeFlowTravelTime,confidence,roadClosure,latitude,longitude,Day,Hour,minute,Delay,delay ratio
0,18,0,39,39,918,918,1.0,0,19.2500,73.0500,27,18,30,0,0.0
1,18,3,23,23,190,190,1.0,0,19.2500,72.9800,27,18,30,0,0.0
2,4,3,6,10,6,3,1.0,0,19.2183,72.9781,27,18,30,3,1.0
3,4,3,19,19,148,148,1.0,0,19.0760,72.8777,27,18,30,0,0.0
4,9,3,21,21,228,228,1.0,0,19.1960,72.9600,27,18,30,0,0.0


In [43]:
df.describe()

,road,frc,currentSpeed,freeFlowSpeed,currentTravelTime,freeFlowTravelTime,confidence,roadClosure,latitude,longitude,Day,Hour,minute,Delay,delay ratio
count,29446.000000,29446.000000,29446.000000,29446.000000,29446.000000,29446.000000,29446.000000,29446.000000,29446.000000,29446.000000,29446.000000,29446.000000,29446.000000,29446.000000,29446.000000
mean,10.266250,2.389595,23.123582,25.810636,962.381138,838.881885,0.957961,0.024791,19.243189,73.072542,10.635808,17.403824,24.851593,123.499253,0.168511
std,6.119544,1.147125,11.693907,11.631899,764.763531,661.120820,0.092805,0.155491,0.217646,0.196642,8.614260,3.514476,16.829180,215.610241,0.286391
min,0.000000,0.000000,2.000000,9.000000,3.000000,3.000000,0.500000,0.000000,18.650000,72.840000,1.000000,11.000000,0.000000,0.000000,0.000000
25%,5.000000,2.000000,15.000000,18.000000,243.000000,220.000000,0.983075,0.000000,19.196000,72.978100,5.000000,14.000000,12.000000,0.000000,0.000000
50%,11.000000,3.000000,20.000000,23.000000,928.000000,833.000000,1.000000,0.000000,19.230000,73.050000,8.000000,17.000000,22.000000,0.000000,0.000000
75%,16.000000,3.000000,29.000000,31.000000,1302.000000,1109.000000,1.000000,0.000000,19.250000,73.100000,12.000000,20.000000,40.000000,170.000000,0.283105
max,20.000000,4.000000,61.000000,61.000000,3862.000000,2552.000000,1.000000,1.000000,20.011000,73.790000,31.000000,23.000000,59.000000,2109.000000,14.070796


In [45]:
def categorize_traffic(delay_ratio):
    if delay_ratio <= 0.15:
        return 'Low'
    elif delay_ratio <= 0.5:
        return 'Medium'
    else:
        return 'High'

df['Traffic level'] = df['delay ratio'].apply(categorize_traffic)



In [48]:
# traffic_data = df.to_csv('traffic_dataset.csv',index=False)
df.head(20)

,road,frc,currentSpeed,freeFlowSpeed,currentTravelTime,freeFlowTravelTime,confidence,roadClosure,latitude,longitude,Day,Hour,minute,Delay,delay ratio,Traffic level
0,18,0,39,39,918,918,1.000000,0,19.2500,73.0500,27,18,30,0,0.000000,Low
1,18,3,23,23,190,190,1.000000,0,19.2500,72.9800,27,18,30,0,0.000000,Low
2,4,3,6,10,6,3,1.000000,0,19.2183,72.9781,27,18,30,3,1.000000,High
3,4,3,19,19,148,148,1.000000,0,19.0760,72.8777,27,18,30,0,0.000000,Low
4,9,3,21,21,228,228,1.000000,0,19.1960,72.9600,27,18,30,0,0.000000,Low
5,9,1,54,54,126,126,1.000000,0,19.0500,72.8700,27,18,30,0,0.000000,Low
6,20,3,32,32,1022,1022,1.000000,0,19.1800,73.0100,27,18,30,0,0.000000,Low
7,20,3,20,20,220,220,1.000000,0,19.0450,73.0150,27,18,30,0,0.000000,Low
8,19,3,15,19,1000,789,0.947993,0,19.0600,72.8850,27,18,30,211,0.267427,Medium
9,19,3,25,30,555,462,0.985374,0,18.9800,73.1100,27,18,30,93,0.201299,Medium
